In [1]:
import csv
from collections import Counter
import collections
import numpy as np
import operator
import h5py
import argparse
import sys
import re
import codecs

In [2]:
FILE_PATHS = {"PTB": ("data/train.txt",
                      "data/valid_blanks.txt",
                      "data/test_blanks.txt",
                      "data/valid_kaggle.txt",
                      "data/valid.txt",
                      "data/words.dict")}
train_path, valid_blanks_path, test_blanks_path, valid_kaggle_path, valid_path, words_path = FILE_PATHS["PTB"]

In [31]:
train = list(csv.reader(open(train_path, 'rb'), delimiter=' '))
valid = list(csv.reader(open(valid_path, 'rb'), delimiter=' '))
valid_blanks = list(csv.reader(open(valid_blanks_path, 'rb'), delimiter=' '))
valid_kaggle = np.array((list(csv.reader(open(valid_kaggle_path, 'rb'), delimiter=',')))[1:]).astype(int)
valid_kaggle = (valid_kaggle[:,1:]).tolist()
valid_kaggle = np.array([valid_kaggle[i].index(1) for i in range(0,len(valid_kaggle))])
test_blanks = list(csv.reader(open(test_blanks_path, 'rb'), delimiter=' '))
word_list = list(csv.reader(open(words_path, 'rb'), delimiter='\t'))
V = len(word_list)+1
word_dict = {}





#input window length here!
window_length = 5








for word in word_list:
    word_dict[word[1]]=int(word[0])
for i in range(0,len(valid)):
    valid[i] = (['<s>']*(window_length-1)) + valid[i][1:-1] + ['</s>']
for i in range(0,len(train)):
    train[i] = (['<s>']*(window_length-1)) + train[i][1:-1] + ['</s>']

In [4]:
def gen_windows(sentences, win_len):
    windows = []
    outputs = []
    for sentence in sentences:
       
        for i in range(win_len, len(sentence)-1):
            window = np.zeros(win_len)

            for j in range(0,win_len):          
                window[j] = word_dict[sentence[i-win_len+j+1]]
            
            windows.append(window)
            outputs.append(word_dict[sentence[i+1]])
    windows = np.array(windows).astype(int)
    outputs = np.array(outputs).astype(int)
    
    return windows, outputs

def q_converter(bag):
    bag_int = np.zeros(len(bag))
    for i in range(0,len(bag_int)):
        bag_int[i] = word_dict[bag[i]]
    return bag_int.astype(int)

def get_last_window(context, win_len):
    context = ['<s>']*(win_len-1)+context
    window = np.zeros(win_len)
    for i in range(0,win_len):
        window[i] = word_dict[context[0-win_len+i]]
    return window

def prediction_window(blanks, win_len):
    valid_input = []
    valid_q = []
    for element in valid_blanks:
        if element[0] == 'Q':
            valid_q.append(q_converter(element[1:]))
        else:
            valid_input.append(get_last_window(element[1:], win_len))
    valid_input = np.array(valid_input).astype(int)
    valid_q = np.array(valid_q).astype(int)
    return valid_input, valid_q

In [5]:
valid_input,valid_q = prediction_window(valid_blanks, window_length)
test_input,test_q = prediction_window(valid_blanks, window_length)
train_input, train_output = gen_windows(train, window_length)

In [6]:
filename = 'data' + '.hdf5'
with h5py.File(filename, "w") as f:
    f['train_input'] = train_input
    f['train_output'] = train_output
    f['valid_input'] = valid_input
    f['valid_q'] = valid_q
    f['test_input'] = test_input
    f['test_q'] = test_q
    f['nfeatures'] = np.array([V], dtype=np.int32)
    f['nclasses'] = np.array([50], dtype=np.int32)

In [42]:
valid_input

array([[ 328, 2517,    7,    1,  660],
       [   3,    3,    2,    2, 2975],
       [  77,  423,  197, 3884,    6],
       ..., 
       [   1,   39,   11,  471,  158],
       [   9,  417,    5,  329, 2040],
       [   5,   23,   38,   14,    5]])

In [39]:
len(_37)

3370

In [40]:
len(valid_blanks)

6740